In [1]:
using Plots
using QuadGK
using NLopt
using SpecialFunctions
using NLsolve
include("../../scaling_CGS.jl")

print_all_CGS (generic function with 2 methods)

In [2]:
#---------- Two Steam Setup ----------
base = BaseUnits("CGS")

electron_mass_scaling = 1e0
charge_scaling = 1e0
eps_0_scaling = 1e0
mu_0_scaling = 1e0

scale_base_units(base, eps_0_scaling, mu_0_scaling, charge_scaling, electron_mass_scaling)

mass_density_scale = 1.673532836356e-14
length_scale       = 227.71078654314857
time_scale         = 1.1137516851221212e-9



length= 227.71078654314857
number_density=1e10
B_flux=10
temperature   = 59298985.44815759
temperature_e = temperature
temperature_p = temperature



scaling = ScalingCGS(base,
                number_density, length, temperature, temperature_e, temperature_p, B_flux,
                length_scale, mass_density_scale, time_scale)

ds = 0.0025
per_cell = 125

set_macro_particle_weights(scaling, ds, per_cell);
#print_all_CGS(scaling)

1.1807305771571771e17

In [3]:
# Penrose criterion function
#   Note, this is based only on electrons and protons are neglected
function Penrose_criterion(U0, V0, v_t)
    function integrand(u)
        p1 = exp(- (u + V0)^2 / (2 * v_t^2))
        p2 = exp(- (u - V0)^2 / (2 * v_t^2))
        p3 = exp(- (U0 + V0)^2 / (2 * v_t^2))
        p4 = exp(- (U0 - V0)^2 / (2 * v_t^2))
        return (p1 + p2 - p3 - p4) / (u - U0)^2
    end

    integral, error = quadgk(integrand, -c*100, c*100) # Multiplying with 10 as in Python code
    return integral, error
end

Penrose_criterion (generic function with 1 method)

In [6]:
#---------- Find unstable limit for "real" physical values --------------

c = scaling.c_real
m_e = scaling.m_e_real

k_B = scaling.k_B_real

V_0 = scaling.c_real * 0.2

U0 = 1e-6 # #Should be zero but integrals crash so just make it a small value compared to the velocities 


#---------- Find zero crossing  ------------------------------
V_t_arr  = collect(range(0.7,stop=0.8, length=10000)) * V_0
Pen_res = zeros(size(V_t_arr))

for (i,v_t) in enumerate(V_t_arr)
    res, err = Penrose_criterion.(U0, V_0, v_t)
    Pen_res[i] = res
end

min_index = argmin(abs.(Pen_res))
v_t_limit = V_t_arr[min_index]
#------------------------------------------------------------

println("Setup is no longer unstable if thermal velocity reaches $(@sprintf("%.2e", v_t_limit)) m/s")
println("Relative to drift this is $(@sprintf("%.5f", v_t_limit / V_0))")
println("This corresponds to a temperature of: $(@sprintf("%.2e", v_t_limit^2 * m_e / k_B)) K")


Setup is no longer unstable if thermal velocity reaches 4.59e+09 m/s
Relative to drift this is 0.76555
This corresponds to a temperature of: 1.39e+08 K


In [7]:
#---------- Find unstable limit for scaled code values --------------

c = scaling.c_code

m_e = scaling.m_e_code

k_B = scaling.k_B_code

V_0 = scaling.c_code * 0.2

U0 = 1e-15 #Should be zero but integrals crash so just make it a small value compared to the velocities 

#---------- Find zero crossing based ------------------------------
V_t_arr  = collect(range(0.7,stop=0.8, length=10000)) * V_0
Pen_res = zeros(size(V_t_arr))

for (i,v_t) in enumerate(V_t_arr)
    res, err = Penrose_criterion.(U0, V_0, v_t)
    #println(i)
    Pen_res[i] = res
end

min_index = argmin(abs.(Pen_res))
v_t_limit = V_t_arr[min_index]
#------------------------------------------------------------

println("Setup is no longer unstable if thermal velocity reaches $(@sprintf("%.2e", v_t_limit)) [ code units ]")
println("Relative to drift this is $(@sprintf("%.5f", v_t_limit / V_0))")
println("This corresponds to a temperature of: $(@sprintf("%.2e", v_t_limit^2 * m_e / k_B)) K")


#Note - the ratio between v_t_limit and v_0 should be the same as the real units 


Setup is no longer unstable if thermal velocity reaches 2.25e-02 [ code units ]
Relative to drift this is 0.76555
This corresponds to a temperature of: 1.39e+08 K
